In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))


DATA_PATH = os.getenv('DATA_PATH')
print(DATA_PATH)


## Reading tfrecords

In [ ]:
filpaths=[]
for root,dirs,files in os.walk(os.path.join(DATA_PATH, "monet_tfrec")):
    for f in files:
       filpaths.append(os.path.join(root,f))

In [ ]:
IMAGE_SIZE = [256, 256]

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    return image

def read_tfrecord(example):
    tfrecord_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

In [ ]:
def load_dataset(filenames, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset

In [ ]:
datasetMonet = load_dataset(filpaths, labeled=True)

## Tensorflow

In [ ]:
%load_ext tensorboard


In [ ]:
from tensorflow.keras import layers


def createGenerator():
    inputs = tf.keras.Input(shape=(1024,))
    # x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, name="embedding")(inputs)

    # x=layers.Dense(16*16*4)(inputs)
    # x=layers.LeakyReLU()(x)
    # x=layers.BatchNormalization()(x)
    x=layers.Reshape((16,16,4))(inputs)

    x=layers.Conv2DTranspose(8, 5, padding="same")(x)
    x=layers.LeakyReLU()(x)
    x=layers.BatchNormalization()(x)
    
    x=layers.Conv2DTranspose(16, 5, strides=(2,2), padding="same")(x)
    x=layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x=layers.Conv2DTranspose(16, 5, padding="same")(x)
    x=layers.LeakyReLU()(x)
    x=layers.BatchNormalization()(x)
    
    x=layers.Conv2DTranspose(32, 5, strides=(2,2), padding="same")(x)
    x=layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x=layers.Conv2DTranspose(32, 5, padding="same")(x)
    x=layers.LeakyReLU()(x)
    x=layers.BatchNormalization()(x)
    
    x=layers.Conv2DTranspose(64, 5, strides=(2,2), padding="same")(x)
    x=layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x=layers.Conv2DTranspose(64, 5, padding="same")(x)
    x=layers.LeakyReLU()(x)
    x=layers.BatchNormalization()(x)

    x=layers.Conv2DTranspose(64, 5, strides=(2,2), padding="same")(x)
    x=layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x=layers.Conv2DTranspose(64, 5, padding="same")(x)
    x=layers.LeakyReLU()(x)
    x=layers.BatchNormalization()(x)
    
    x=layers.Conv2DTranspose(3, 5, padding="same")(x)
    outputs = tf.keras.activations.tanh(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="generator")

generator = createGenerator()

generator.summary()


In [ ]:
from tensorflow.keras import layers


def createDiscriminator():
    inputs = tf.keras.Input(shape=(*IMAGE_SIZE, 3))
    # x=layers.Embedding(VOCAB_SIZE, EMBED_DIM, name="embedding")(inputs)
    # x = layers.BatchNormalization()(tf.expand_dims(inputs,2))

    x = layers.Conv2D(8,5)(inputs)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(8,5)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(16,5, strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(16,5)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32,5, strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32,5)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32,5, strides=2)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32,5)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32,5)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(32,5)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)

    x = layers.Flatten()(x)

    x=layers.Dense(32)(x)
    x = layers.LeakyReLU()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.1)(x)

    x=layers.Dense(32)(x)
    x = layers.LeakyReLU()(x)

    outputs=layers.Dense(1, activation=tf.keras.activations.sigmoid)(x)

    return tf.keras.Model(inputs=inputs, outputs=outputs, name="Discriminator")

discriminator = createDiscriminator()

discriminator.summary()


In [ ]:
def saveImages(model, epoch):
  testInput = tf.random.uniform((9, 1024))
  predictions = model(testInput, training=False)

  fig = plt.figure(figsize=(5, 5))

  for i in range(predictions.shape[0]):
    plt.subplot(3, 3, i+1)
    plt.imshow(np.rint(predictions[i, :, :, :] * 127.5 + 127.5).astype(int))
    plt.axis('off')

  plt.savefig(os.path.join(DATA_PATH,"trainImages", 'epoch_{:04d}.png'.format(epoch)))

In [ ]:

BATCH_SIZE=16
LOG_INTERVAL=5
epochs = 15
saveModel=False


log_dir = "./logs/"+generator.name+"/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,
                                                      write_graph=True, update_freq=5)


summary_writer = tf.summary.create_file_writer(log_dir)

# Instantiate an optimizer .
# optimizer = tf.keras.optimizers.SGD(learning_rate=lr_schedule)
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
optimizerGen = tf.keras.optimizers.Adam(learning_rate=1e-3)
optimizerDis = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Instantiate a loss function.
lossFnGen = tf.keras.losses.BinaryCrossentropy()
lossFnDis = tf.keras.losses.BinaryCrossentropy()

accuracyDis = tf.keras.metrics.BinaryAccuracy()

batchedDataset = datasetMonet.batch(BATCH_SIZE, drop_remainder=False)

@tf.function()
def trainStepGen(trueImages, trainGen=True, trainDis=True):
    input = tf.random.uniform((BATCH_SIZE, 1024))
    with tf.GradientTape() as tapeGen, tf.GradientTape() as tapeDis:
        fakeImages = generator(input, training=True) 

        discOutputFake = discriminator(fakeImages, training=True)
        discOutputTrue = discriminator(trueImages, training=True)

        lossGen = lossFnGen(tf.ones_like(discOutputFake), discOutputFake)

        lossDisTrue = lossFnDis(tf.ones_like(discOutputTrue), discOutputTrue)
        lossDisFake = lossFnDis(tf.zeros_like(discOutputFake), discOutputFake)
        totalLossDis = lossDisTrue + lossDisFake

    gradsGen = tapeGen.gradient(lossGen, generator.trainable_weights)
    gradsDis = tapeDis.gradient(totalLossDis, discriminator.trainable_weights)

    accuracyDis.update_state(tf.zeros_like(discOutputFake), discOutputFake)
    accuracyDis.update_state(tf.ones_like(discOutputTrue), discOutputTrue)

    if trainGen:
        optimizerGen.apply_gradients(zip(gradsGen, generator.trainable_weights)) 
    if trainDis:
        optimizerDis.apply_gradients(zip(gradsDis, discriminator.trainable_weights)) 

    return lossGen, totalLossDis



maxStep=0

for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch+1,))

    # Iterate over the batches of the dataset.
    for step, x_batch_train in enumerate(batchedDataset):

        if step%3==0:
            lossGen, lossDis = trainStepGen(x_batch_train, trainDis=True)
        else:
            lossGen, lossDis = trainStepGen(x_batch_train, trainDis=False)

        # Log 
        if step % LOG_INTERVAL == 0:
            template = 'Epoch {}/Step {}, Loss Generator: {:.4f}, Loss Discriminator: {:.4f}, Accuracy Dis: {:.4f}'
            print(template.format(epoch+1, step, lossGen.numpy(), lossDis.numpy(),  accuracyDis.result().numpy()))
            
            with summary_writer.as_default():
                tf.summary.scalar('lossGen', lossGen, step=maxStep*epoch+step)
                tf.summary.scalar('lossDis', lossDis, step=maxStep*epoch+step)
                tf.summary.scalar('Disc Accuracy', accuracyDis.result().numpy(), step=maxStep*epoch+step)
                summary_writer.flush()

    maxStep=step

    accuracyDis.reset_state()

    print("Saving images")
    saveImages(generator, epoch)

    if saveModel:
      generator.save(os.path.join(DATA_PATH, "model_"+"_epoch_{}_valF1Score{:.3f}".format(epoch, float(val_f1))))

In [ ]:
testInput = tf.random.uniform((9, 1024))
predictions = generator(testInput, training=False)

fig = plt.figure(figsize=(5, 5))

for i in range(predictions.shape[0]):
    plt.subplot(3, 3, i+1)
    plt.imshow(np.rint(predictions[i, :, :, :] * 127.5 + 127.5).astype(int))
    plt.axis('off')

plt.show()